Lab9

---

# Contextual Question Answering
## Abstractive QA

The aim of this exercise is building a neural model able to answer contextual questions in the legal domain.

Training and Validation dataset: PoQuAD

Testing dataset: Simple Legal Questions Dataset

Resources:
https://medium.com/@ajazturki10/simplifying-language-understanding-a-beginners-guide-to-question-answering-with-t5-and-pytorch-253e0d6aac54



In [1]:
!pip install transformers evaluate rouge wandb nltk sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 615.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

In [12]:
TOKENIZER = T5TokenizerFast.from_pretrained("allegro/plt5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("allegro/plt5-base", return_dict=True)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)
Q_LEN = 256   # Question Length
T_LEN = 32    # Target Length
BATCH_SIZE = 16
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

## Download PoQuAD Dataset (Training and Validation)

In [3]:
#for AQA
!wget https://huggingface.co/datasets/clarin-pl/poquad/resolve/main/poquad-train.json
!wget https://huggingface.co/datasets/clarin-pl/poquad/resolve/main/poquad-dev.json

--2025-01-04 22:51:57--  https://huggingface.co/datasets/clarin-pl/poquad/resolve/main/poquad-train.json
Resolving huggingface.co (huggingface.co)... 65.8.243.46, 65.8.243.16, 65.8.243.90, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.46|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/18/de/18ded45e8046dd5f58b7365947f5a4298433a0e7710248308670e8cf26059c20/b1ac3acabb49fedb7bb7db0de0690ddb22585d6419321589cc1bb0a8068a4ff9?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27poquad-train.json%3B+filename%3D%22poquad-train.json%22%3B&response-content-type=application%2Fjson&Expires=1736290317&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNjI5MDMxN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8xOC9kZS8xOGRlZDQ1ZTgwNDZkZDVmNThiNzM2NTk0N2Y1YTQyOTg0MzNhMGU3NzEwMjQ4MzA4NjcwZThjZjI2MDU5YzIwL2IxYWMzYWNhYmI0OWZlZGI3YmI3ZGIwZGUwNjkwZGRiMjI1ODVkNjQxOTMyMTU4OWNjMWJiMG

In [4]:
with open('poquad-train.json') as f:
    train_data = json.load(f)
with open('poquad-dev.json') as f:
    validation_data = json.load(f)

In [5]:
print("CONTEXT\n", train_data['data'][0]['paragraphs'][0]['context'])
print("QESTION\n", train_data['data'][0]['paragraphs'][0]['qas'][0]['question'])
print("ANSWER\n", train_data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['generative_answer'])

CONTEXT
 Projekty konfederacji zaczęły się załamywać 5 sierpnia 1942. Ponownie wróciła kwestia monachijska, co uaktywniło się wymianą listów Ripka – Stroński. Natomiast 17 sierpnia 1942 doszło do spotkania E. Beneša i J. Masaryka z jednej a Wł. Sikorskiego i E. Raczyńskiego z drugiej strony. Polscy dyplomaci zaproponowali podpisanie układu konfederacyjnego. W następnym miesiącu, tj. 24 września, strona polska przesłała na ręce J. Masaryka projekt deklaracji o przyszłej konfederacji obu państw. Strona czechosłowacka projekt przyjęła, lecz już w listopadzie 1942 E. Beneš podważył ideę konfederacji. W zamian zaproponowano zawarcie układu sojuszniczego z Polską na 20 lat (formalnie nastąpiło to 20 listopada 1942).
QESTION
 Co było powodem powrócenia konceptu porozumieniu monachijskiego?
ANSWER
 wymiana listów Ripka – Stroński


### Preprocessing

In [6]:
# Extracting context, question, and answers from the dataset

def prepare_data(data):
    articles = []

    for article in data["data"]:
        for paragraph in article["paragraphs"]:
            for qa in paragraph["qas"]:
                question = qa["question"]

                if not qa["is_impossible"]:
                  answer = qa["answers"][0]["text"]

                inputs = {"context": paragraph["context"], "question": question, "answer": answer}


                articles.append(inputs)

    return articles

train_data = prepare_data(train_data)
validation_data = prepare_data(validation_data)

# Create a Dataframe
train_data = pd.DataFrame(train_data)
validation_data = pd.DataFrame(validation_data)

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]

        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = TOKENIZER.pad_token_id

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [ ]:
# Dataloader

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(validation_data.index)

data =  pd.concat([train_data, validation_data], axis=0, ignore_index=True)
qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [ ]:
print("Training batches: ", len(train_loader))
print("Validation batches: ", len(val_loader))

Training batches:  7078
Validation batches:  883


# Training

In [ ]:
import wandb
wandb.init(project="PJN9", name="athena_plt5-base_poquad")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
train_loss = 0.0
val_loss = 0.0
train_bleu_score = 0.0
val_bleu_score = 0.0
train_batch_count = 0
val_batch_count = 0
save_step = 300
log_step = 100
bleu_step = 100

bleu_metric = evaluate.load("google_bleu")
MODEL.to(DEVICE)

for epoch in range(3):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1

        if train_batch_count  % bleu_step == 0:
          with torch.no_grad():
            outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)

            # Convert outputs and labels to strings
            predictions = [TOKENIZER.decode(pred, skip_special_tokens=True) for pred in outputs]
            references = [TOKENIZER.decode(label, skip_special_tokens=True) for label in labels]

            # Compute BLEU scores
            for pred, ref in zip(predictions, references):
                bleu_score = bleu_metric.compute(predictions=[pred], references=[[ref]])
                train_bleu_score += bleu_score["google_bleu"]

        if train_batch_count % save_step == 0:
          MODEL.save_pretrained("qa_model")
          TOKENIZER.save_pretrained("qa_tokenizer")

        if train_batch_count  % log_step == 0 :
          print(f"{epoch+1}/{2} -> Train loss: {train_loss }")
          wandb.log({"train_loss": train_loss / train_batch_count, "train_bleu": train_bleu_score / train_batch_count})

    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1

        if val_batch_count  % bleu_step == 0:
          with torch.no_grad():
            outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)

            # Convert outputs and labels to strings
            predictions = [TOKENIZER.decode(pred, skip_special_tokens=True) for pred in outputs]
            references = [TOKENIZER.decode(label, skip_special_tokens=True) for label in labels]

            # Compute BLEU scores
            for pred, ref in zip(predictions, references):
                bleu_score = bleu_metric.compute(predictions=[pred], references=[[ref]])
                val_bleu_score += bleu_score["google_bleu"]

        if val_batch_count  % log_step == 0 :
          print(f"{epoch+1}/{2} -> Validation loss: {val_loss / val_batch_count }")
          wandb.log({"val_loss": val_loss / val_batch_count, "val_bleu": val_bleu_score / val_batch_count})


Training batches:   1%|▏         | 100/7078 [00:27<4:37:20,  2.38s/it]

1/2 -> Train loss: 12360.009284973145


Training batches:   3%|▎         | 201/7078 [00:43<30:46,  3.73it/s]  

1/2 -> Train loss: 22501.7628326416


Training batches:   4%|▍         | 301/7078 [01:01<1:12:49,  1.55it/s]

...

Validation batches:  83%|████████▎ | 735/883 [01:57<00:37,  3.93it/s]

3/2 -> Validation loss: 1.0774873459994794


Validation batches:  95%|█████████▍| 835/883 [02:13<00:12,  3.71it/s]

3/2 -> Validation loss: 1.048431235546103


Validation batches: 100%|██████████| 883/883 [02:20<00:00,  6.27it/s]


# Example predictions

In [18]:
TOKENIZER = T5TokenizerFast.from_pretrained("qa_tokenizer")
MODEL = T5ForConditionalGeneration.from_pretrained("qa_model")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer and ref_answer != '':
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer],
                            references=[ref_answer])
        return {
            "ref_answer": ref_answer,
            "predicted_answer": predicted_answer,
            "bleu_score": score['google_bleu']
        }
    else:
       return {
            "ref_answer": None,
            "predicted_answer": predicted_answer,
            "bleu_score": None
        }
def print_prediction(context, question, pred):
  print("Context: \n", context)
  print("Question: \n", question)
  print("\nReference Answer: ", pred['ref_answer'])
  print("Predicted Answer: ", pred['predicted_answer'])
  print("BLEU Score: ", round(pred['bleu_score'], 3))

In [17]:
!cp /content/drive/MyDrive/poquad/model.safetensors qa_model/

#### PoQuAD Train Set

In [19]:
context = train_data.iloc[20]["context"]
question = train_data.iloc[20]["question"]
ref_answer = train_data.iloc[20]["answer"]
pred = predict_answer(context, question, ref_answer)
print_prediction(context, question, pred)
print("\n\n")

context = train_data.iloc[40]["context"]
question = train_data.iloc[40]["question"]
ref_answer = train_data.iloc[40]["answer"]
pred = predict_answer(context, question, ref_answer)
print_prediction(context, question, pred)
print("\n\n")

context = train_data.iloc[60]["context"]
question = train_data.iloc[60]["question"]
ref_answer = train_data.iloc[60]["answer"]
pred = predict_answer(context, question, ref_answer)
print_prediction(context, question, pred)
print("\n\n")

Context: 
 W sezonie 1959 nowym szkoleniowcem Górnika został węgierski trener János Steiner, z którym Kowal współpracował w Legii. Kowal wystąpił we wszystkich dwudziestu dwóch meczach ligowych i zdobył sześć bramek, spędzając na boisku 1935 minut. Jedynym spotkaniem, którego nie dograł w pełnym wymiarze czasowym był mecz przeciwko Ruchowi Chorzów (2:2, 25 października 1959 roku), kiedy to został zmieniony przez Manfreda Fojcika. Kowal był odpowiedzialny za wykonywanie rzutów karnych, które wykorzystał w spotkaniach z Legią (1:2, 10 maja 1959 roku) oraz Lechią Gdańsk (3:2, 1 listopada 1959 roku). Bramkarza Lechii pokonał nie biorąc rozbiegu w kierunku piłki. Przed strzałem wykonał zwód, po którym Henryk Gronowski rzucił się w jeden róg bramki, a piłka potoczyła się w drugi. Górnik zapewnił sobie tytuł mistrzowski na trzy kolejki przez zakończeniem rozgrywek, natomiast Kowal był uważany za piłkarza w szczytowej formie oraz inteligentnego konstruktora akcji.
Question: 
 Z jakiego kraju p

#### PoQuAD Validation Set

In [20]:
context = validation_data.iloc[20]["context"]
question = validation_data.iloc[20]["question"]
ref_answer = validation_data.iloc[20]["answer"]
pred = predict_answer(context, question, ref_answer)
print_prediction(context, question, pred)
print("\n\n")

context = validation_data.iloc[40]["context"]
question = validation_data.iloc[40]["question"]
ref_answer = validation_data.iloc[40]["answer"]
pred = predict_answer(context, question, ref_answer)
print_prediction(context, question, pred)
print("\n\n")

context = validation_data.iloc[60]["context"]
question = validation_data.iloc[60]["question"]
ref_answer = validation_data.iloc[60]["answer"]
pred = predict_answer(context, question, ref_answer)
print_prediction(context, question, pred)
print("\n\n")

Context: 
 Od 2014 roku w Superpucharze, z inicjatywy prezesa PZPN Zbigniewa Bońka, nastąpiła zmiana, gdyż po 8 latach przerwy związek podjął na swoje barki organizację rozgrywek o Superpuchar. O trofeum walczyć będą Mistrz Polski oraz zdobywca Pucharu Polski sezonu zakończonego w roku rozgrywania Superpucharu. Tym samym powrócono do nazwy Superpuchar Polski. Mecz rozgrywany będzie na stadionie Mistrza Polski, na około tydzień przed startem nowego sezonu Ekstraklasy. W sytuacji, gdy ten sam klub sięgnie po mistrzostwo i Puchar Polski, jego rywalem w meczu o trofeum będzie finalista ostatniej edycji Pucharu Polski.
Question: 
 Kto piastował stanowisko prezesa Polskiego Związku Piłki Nożnej poczynając od 2014 roku?

Reference Answer:  pączki, faworki i bliny
Predicted Answer:  na stadionie Mistrza Polski
BLEU Score:  0.0



Context: 
 Eminem używa różnych tożsamości w swoich piosenkach, by korzystać z różnych stylów rapowania i różnych podmiotów. Jego najbardziej znane i popularne alter 

## Evaluation on Simple Legal Questions Dataset

### Loading dataset from json files

In [ ]:
import json

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

answers = load_jsonl('answers.jl')
questions = load_jsonl('questions.jl')
passages = load_jsonl('passages.jl')
relevant = load_jsonl('relevant.jl')

In [ ]:
print(answers[0])

{'score': '1', 'question-id': '1', 'answer': 'Tak, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3.'}


In [ ]:
test_set = []

for row in relevant:
  context_id = row["passage-id"]
  question_id = row["question-id"]

  for passage in passages:
    if passage["_id"] == context_id:
      context_text = passage["text"]
      break
  question_text = questions[int(question_id)-1]["text"]

  for answer in answers:
    if answer["question-id"] == row["question-id"]:
      answer_text = answer["answer"]
      break

  test_set.append({
      "context": context_text,
      "question": question_text,
      "answer": answer_text
  })


In [ ]:
print("Simple Legal Ouestions Test Set Example\n")
print("context: ", test_set[0]['context'])
print("question: ",test_set[0]['question'])
print("answer: ", test_set[0]['answer'])

Simple Legal Ouestions Test Set Example

context:  Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełożonego, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3. § 2. Jeżeli sprawca dopuszcza się czynnej napaści w związku z pełnieniem przez przełożonego obowiązków służbowych albo wspólnie z innymi żołnierzami lub w obecności zebranych żołnierzy, podlega karze pozbawienia wolności od 6 miesięcy do lat 8. § 3. Jeżeli sprawca czynu określonego w § 1 lub 2 używa broni, noża lub innego podobnie niebezpiecznego przedmiotu, podlega karze pozbawienia wolności od roku do lat 10. § 4. Karze przewidzianej w § 3 podlega sprawca czynu określonego w § 1 lub 2, jeżeli jego następstwem jest skutek określony w art. 156 lub 157 § 1.
question:  Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega karze pozbawienia wolności?
answer:  Tak, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3.


### Example Predictions with BLEU score

#### Simple Legal Questions Dataset

In [ ]:
context = test_set[20]["context"]
question = test_set[20]["question"]
ref_answer = test_set[20]["answer"]
pred = predict_answer(context, question, ref_answer)
print_prediction(context, question, pred)
print("\n\n")

context = test_set[40]["context"]
question = test_set[40]["question"]
ref_answer = test_set[40]["answer"]
pred = predict_answer(context, question, ref_answer)
print_prediction(context, question, pred)
print("\n\n")

context = test_set[60]["context"]
question = test_set[60]["question"]
ref_answer = test_set[60]["answer"]
pred = predict_answer(context, question, ref_answer)
print_prediction(context, question, pred)
print("\n\n")


Context: 
 Art. 209. § 1. Kto uporczywie uchyla się od wykonania ciążącego na nim z mocy ustawy lub orzeczenia sądowego obowiązku opieki przez niełożenie na utrzymanie osoby najbliższej lub innej osoby i przez to naraża ją na niemożność zaspokojenia podstawowych potrzeb życiowych, podlega grzywnie, karze ograniczenia wolności albo pozbawienia wolności do lat 2. § 2. Ściganie następuje na wniosek pokrzywdzonego, organu opieki społecznej lub właściwej instytucji. § 3. Jeżeli pokrzywdzonemu przyznano świadczenie z funduszu alimentacyjnego, ściganie odbywa się z urzędu.


Question: 
 Jak ściga się świadczenia w ramach funduszu alimentacyjnego?

Reference Answer:  Ściganie następuje na wniosek pokrzywdzonego, organu opieki społecznej lub właściwej instytucji.
Predicted Answer:  z urzędu
BLEU Score:  0.0



Context: 
 Art. 58. 1. Kontrolujący w toku kontroli może również dokonać przeszukania pomieszczeń lub rzeczy, za zgodą sądu antymonopolowego, udzieloną na wniosek Prezesa Urzędu. Przy pr

### Mean BLEU for whole testing dataset

In [ ]:
scores = []
mean_bleu = 0.0

for i in range(len(test_set)):
  context = test_set[i]["context"]
  question = test_set[i]["question"]
  ref_answer = test_set[i]["answer"]
  pred = predict_answer(context, question, ref_answer)
  if pred['bleu_score'] is not None:
    scores.append(pred['bleu_score'])

mean_bleu = sum(scores) / len(scores)
print("Mean BLEU Score: ", round(mean_bleu, 3))


Mean BLEU Score:  0.056


# Summary

> Does the performance on the validation dataset reflects the performance on your test set?

They are both bad...

>What are the outcomes of the model on your test questions? Are they satisfying? If not, what might be the reason for that?

Most of the answers seem to be to abstracted from context. Either halucinations or inability to work with eg. numbers. Both questions ans the ground truth answers might not be clear or correct even. So some model predictions might be correct but not similar to the ground truth in BLEU measure.


>Why extractive question answering is not well suited for inflectional languages?

The answers to the questions may lack context and might not be correctly lemmatized. The syntax of extracted answer could contain errors.